In [48]:
import pandas as pd
from config import api_key_google
import os

In [49]:
# !pip install --upgrade google-api-python-client
# !pip install google-colab

In [50]:
model_name = "geminipro"
API_KEY = api_key_google

In [16]:
# datasets/Haystack_Needles/Incongruous/mBBC_2024/GPT4/English_needles.csv
dataset_path = "datasets/Haystack_Needles/Incongruous/mBBC_2024/Gemini/English_needles.csv"

eng_needles = pd.read_csv(dataset_path)
eng_needles.head(7)

,haystack_length,depth,haystack_with_needle,needle_city,needle_digit
0,2k,0,The special magic Sarajevo number is: 7032739...,Sarajevo,7032739
1,2k,25,Star dunes - or pyramid dunes - are named afte...,Bangalore,9181353
2,2k,50,Star dunes - or pyramid dunes - are named afte...,Los Angeles,2541112
3,2k,75,Star dunes - or pyramid dunes - are named afte...,Yangon,9965006
4,2k,100,Star dunes - or pyramid dunes - are named afte...,Colombo,9049628
5,8k,0,The special magic Seoul number is: 6401349 St...,Seoul,6401349
6,8k,25,Star dunes - or pyramid dunes - are named afte...,Thimphu,8205926


In [17]:
import requests
import json

url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent"
headers = {
    "Content-Type": "application/json"
}

params = {
    "key": API_KEY 
}

In [ ]:
generated_text_3 = []

for text in eng_needles["haystack_with_needle"][15:]:
    prompt = f"{text}\nHere is the magic number:"
    # prompt = f'''
    #     You are a helpful AI bot that answers questions for a user. Keep your response short and direct.
    #     The following is a set of context and a question that will relate to the context. 
    #     #CONTEXT
    #     {text}
    #     #ENDCONTEXT
        
    #     #QUESTION
    #     What is the special magic {{}} number? Don't give information outside the document or repeat your findings. If the
    #     information is not available in the context respond UNANSWERABLE.
    #     '''
    
    payload = {
        "contents": [{
            "parts": [{
                "text": prompt
            }]
        }],
         "safetySettings": [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}
        ]
    }
    
    
    response = requests.post(url, headers=headers, json=payload, params=params)
    
    data = json.loads(response.text)
    output_text = data['candidates'][0]['content']['parts'][0]['text']

    generated_text_3.append(output_text)

In [ ]:
missed = ['This context does not mention anything about a magic number, so I cannot answer this question from the provided context.']
len(generated_text), len(generated_text_1), len(missed), len(generated_text_2), len(generated_text_3)

In [ ]:
# Evalutaion

def check_label(row):
    if str(row['needle_digit']) in row['generated_text']:
        return 1
    else:
        return 0

eng_needles['generated_text'] = generated_text_ori
eng_needles['found'] = eng_needles.apply(check_label, axis=1)
eng_needles.to_csv(f"outputs/mBBC_2024/{model_name}.csv", index=False)

## 3-Needles Evaluation

In [98]:
import pandas as pd

def evaluate_3needle_v2(df):
    df['bigger_label'] = df.apply(lambda row: max(row['label1'], row['label2'], row['label3']), axis=1)
    df['found'] = df['bigger_label'].astype(str).apply(lambda x: df['generated_text'].str.contains(x).any()).astype(int)
    return df

In [100]:
haystack_lang = "swa"
prompt_lang = "eng"

df = pd.read_csv(f"outputs/Incongruous/3-Needles/mBBC_2024/Gemini/Supporting/gemini-prompts_{prompt_lang}_{haystack_lang}_number.csv")

df = evaluate_3needle_v2(df)
df.to_csv(f"outputs/Incongruous/3-Needles/mBBC_2024/Gemini/gemini_1_5_pro_{haystack_lang}_{prompt_lang}_number.csv", index=False)
df.head()

,context_length,position,text,city1,label1,city2,label2,city3,label3,prompt,generated_text,bigger_label,found
0,2k,0,Nambari maalum ya Almaty ni 1927155. Chanzo c...,Almaty,1927155,Bukarest,8955219,Ho Chi Minh,2225018,You are a helpful AI bot that answers question...,8955219 (Bukarest),8955219,1
1,2k,25,"Chanzo cha picha, University of BathKwa wengin...",Antananarivo,8952451,Amman,1312231,Brussels,6752066,You are a helpful AI bot that answers question...,UNANSWERABLE,8952451,0
2,2k,50,"Chanzo cha picha, University of BathKwa wengin...",Bratislava,5212618,Bukarest,3356056,Yerevan,8205570,You are a helpful AI bot that answers question...,UNANSWERABLE,8205570,0
3,2k,75,"Chanzo cha picha, University of BathKwa wengin...",Amsterdam,9159355,Madrid,3133134,Maputo,6463514,You are a helpful AI bot that answers question...,9159355,9159355,1
4,8k,0,Nambari maalum ya Delhi ni 6595659. Chanzo ch...,Delhi,6595659,Kampala,9998979,Athene,2034198,You are a helpful AI bot that answers question...,9998979 (Kampala),9998979,1


## 2-Needles Evaluation

In [90]:
import pandas as pd
from needle_dictionary import Cities

def evaluate_2needle_v2(df):    
    df['bigger_label'] = df.apply(lambda row: row['label1'] if row['label1'] > row['label2'] else row['label2'], axis=1)
    # df['found'] = df['bigger_label'].astype(str).apply(lambda x: df['generated_text'].str.contains(x).any()).astype(int)
    df['found'] = df['bigger_label'].astype(str).apply(lambda x: df['generated_text'].astype(str).str.contains(x).any()).astype(int)
    return df

In [91]:
haystack_lang = "vie"
prompt_lang = "eng"

df = pd.read_csv(f"outputs/Incongruous/2-Needles/mBBC_2024/Gemini/Supporting/gemini-prompts_{prompt_lang}_{haystack_lang}_number.csv")

df = evaluate_2needle_v2(df)
df.to_csv(f"outputs/Incongruous/2-Needles/mBBC_2024/Gemini/gemini_1_5_pro_{haystack_lang}_{prompt_lang}_number.csv", index=False)
df.head()

,context_length,position,text,city1,label1,city2,label2,prompt,generated_text,bigger_label,found
0,2k,0,Số Toronto ma thuật đặc biệt là 1144147 Nguồn...,Toronto,1144147,Madrid,8186086,You are a helpful AI bot that answers question...,8186086,8186086,1
1,2k,25,"Nguồn hình ảnh, Win McNamee/Getty ImagesCựu T...",Istanbul,9718536,Bucharest,1076717,You are a helpful AI bot that answers question...,9718536,9718536,1
2,2k,50,"Nguồn hình ảnh, Win McNamee/Getty ImagesCựu T...",Melbourne,6076265,Tunis,8335770,You are a helpful AI bot that answers question...,8335770,8335770,1
3,2k,75,"Nguồn hình ảnh, Win McNamee/Getty ImagesCựu T...",Thành phố México,2455301,Lisbon,2115973,You are a helpful AI bot that answers question...,2455301,2455301,1
4,8k,0,Số Barcelona ma thuật đặc biệt là 6260938 Ngu...,Barcelona,6260938,Lima,8104145,You are a helpful AI bot that answers question...,8104145,8104145,1


## 1-Needle Evaluation

In [52]:
# Evalutaion

lang = "vie"
eng_needles = pd.read_csv(f"outputs/Incongruous/1-Needle/mBBC_2024/Gemini/Supporting/gemini-prompts_{lang}_v2.csv")

def check_label(row):
    if str(row['label']) in row['generated_text']:
        return 1
    else:
        return 0

eng_needles['found'] = eng_needles.apply(check_label, axis=1)
eng_needles.to_csv(f"outputs/Incongruous/1-Needle/mBBC_2024/Gemini/gemini_pro_1_5_{lang}_eng_v2.csv", index=False)

## Find Article Evaluation

In [56]:
# Evalutaion

haystack_lang = "eng"
prompt_lang = "eng"
df_articles = pd.read_csv(f"outputs/Find_Articles/Gemini/Supporting/gemini_{haystack_lang}_{prompt_lang}_prompts.csv")

def evaluate(row):
    if str(row['label'][8:]) in row['generated_text']:
        return 1
    else:
        return 0

df_articles['found'] = df_articles.apply(evaluate, axis=1)
df_articles.to_csv(f"outputs/Find_Articles/Gemini/gemini_{haystack_lang}_{prompt_lang}.csv", index=False)